In [1]:
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText
from PIL import Image
import requests
from io import BytesIO



C:\Users\MSI\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\MSI\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
C:\Users\MSI\AppData\Local\Programs\

In [ ]:
# # Set device
# device = "cuda" if torch.cuda.is_available() else "cpu"

# # Load model & processor
# model = AutoModelForImageTextToText.from_pretrained(
#     "stepfun-ai/GOT-OCR-2.0-hf",
#     device_map="auto",  # or use "cuda" with .to(device)
# )
# model.to(device)

# processor = AutoProcessor.from_pretrained("stepfun-ai/GOT-OCR-2.0-hf")

# # Load the image from URL
# image_url = "https://huggingface.co/datasets/hf-internal-testing/fixtures_got_ocr/resolve/main/image_ocr.jpg"
# response = requests.get(image_url)
# image = Image.open(BytesIO(response.content)).convert("RGB")

# # Preprocess
# inputs = processor(image, return_tensors="pt").to(device)

# # Generate text
# generate_ids = model.generate(
#     **inputs,
#     do_sample=False,
#     tokenizer=processor.tokenizer,
#     stop_strings="<|im_end|>",
#     max_new_tokens=4096,
# )

# # Decode output
# output_text = processor.decode(generate_ids[0, inputs["input_ids"].shape[1]:], skip_special_tokens=True)
# print(output_text)


In [5]:
from transformers import AutoProcessor, AutoModelForImageTextToText
print(torch.cuda.is_available())
device = "cuda" if torch.cuda.is_available() else "cpu"
# model = AutoModelForImageTextToText.from_pretrained("stepfun-ai/GOT-OCR-2.0-hf", device_map=device)
# processor = AutoProcessor.from_pretrained("stepfun-ai/GOT-OCR-2.0-hf")
model = AutoModelForImageTextToText.from_pretrained("got_ocr_model", device_map=device)
processor = AutoProcessor.from_pretrained("got_ocr_model")

# image = "https://huggingface.co/datasets/hf-internal-testing/fixtures_got_ocr/resolve/main/image_ocr.jpg"
# inputs = processor(image, return_tensors="pt").to(device)

# generate_ids = model.generate(
#     **inputs,
#     do_sample=False,
#     tokenizer=processor.tokenizer,
#     stop_strings="<|im_end|>",
#     max_new_tokens=4096,
# )

# processor.decode(generate_ids[0, inputs["input_ids"].shape[1]:], skip_special_tokens=True)


True


'R&D QUALITY IMPROVEMENT\nSUGGESTION/SOLUTION FORM\nName/Phone Ext. : M. Hamann, P. Harper, P. Martinez\nDate: 9/3/92\nSupervisor/Manager: J. S. Wigand\nR&D Group: Licensee\nSuggestion:\nDiscontinue coal retention analyses on licensee submitted\nproduct samples. (Note: Coal Retention testing is not\nperformed by most licensees. Other B&W physical\nmeasurements as ends stability and inspection for soft\nspots in cigarettes are thought to be sufficient measures\nto assure cigarette physical integrity. The proposed\naction will increase laboratory productivity.)\nSuggested Solution(s) : Delete coal retention from the list of standard\nanalyses performed on licensee submitted\nproduct samples. Special requests for coal\nretention testing could still be submitted on\nan exception basis.\nHave you contacted your Manager/Supervisor?\nYes\nNo\nManager Comments: Manager, please contact suggester and forward\ncomments to the Quality Council.\nqip.wp\n597005708'

In [3]:
# Save them to local folder
# model.save_pretrained("got_ocr_model")
# processor.save_pretrained("got_ocr_model")

[]

In [14]:
def get_ocr(path):
    image = path
    inputs = processor(image, return_tensors="pt").to(device)
    
    generate_ids = model.generate(
        **inputs,
        do_sample=False,
        tokenizer=processor.tokenizer,
        stop_strings="<|im_end|>",
        max_new_tokens=4096,
    )
    
    return processor.decode(generate_ids[0, inputs["input_ids"].shape[1]:], skip_special_tokens=True)

In [17]:
import tkinter as tk
from PIL import ImageGrab, Image
import pyautogui
import io
import win32clipboard  # Windows only
from PIL import ImageTk
import time

coords = {"x1": 0, "y1": 0, "x2": 0, "y2": 0}
rect_id = None

def on_mouse_down(event):
    coords["x1"] = event.x
    coords["y1"] = event.y
    global rect_id
    rect_id = canvas.create_rectangle(event.x, event.y, event.x, event.y, outline='red', width=2)

def on_mouse_move(event):
    if rect_id:
        canvas.coords(rect_id, coords["x1"], coords["y1"], event.x, event.y)

def on_mouse_up(event):
    coords["x2"] = event.x
    coords["y2"] = event.y
    root.destroy()

def image_to_clipboard(img: Image.Image):
    output = io.BytesIO()
    img.convert("RGB").save(output, "BMP")
    data = output.getvalue()[14:]  # Skip BMP header
    output.close()

    win32clipboard.OpenClipboard()
    win32clipboard.EmptyClipboard()
    win32clipboard.SetClipboardData(win32clipboard.CF_DIB, data)
    win32clipboard.CloseClipboard()

# --- UI setup ---
time.sleep(0.5)
root = tk.Tk()
root.attributes("-alpha", 0.3)
root.attributes("-fullscreen", True)
root.attributes("-topmost", True)
root.configure(bg='gray')

canvas = tk.Canvas(root, cursor="cross", bg='gray')
canvas.pack(fill=tk.BOTH, expand=True)

canvas.bind("<ButtonPress-1>", on_mouse_down)
canvas.bind("<B1-Motion>", on_mouse_move)
canvas.bind("<ButtonRelease-1>", on_mouse_up)

print("🔲 Drag to select screen area...")
root.mainloop()

# Get coordinates
x1, y1 = coords["x1"], coords["y1"]
x2, y2 = coords["x2"], coords["y2"]
x1, x2 = sorted([x1, x2])
y1, y2 = sorted([y1, y2])
width, height = x2 - x1, y2 - y1

# Screenshot selected area
screenshot = pyautogui.screenshot(region=(x1, y1, width, height))
screenshot.save("snipped_area.png")
image_to_clipboard(screenshot)

print("✅ Snip saved as 'snipped_area.png'")
print("📋 Image copied to clipboard!")
get_ocr("snipped_area.png")

🔲 Drag to select screen area...
✅ Snip saved as 'snipped_area.png'
📋 Image copied to clipboard!


'JupyterLab Python3(ipykernel)'